In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
fashion_mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

In [ ]:
range_start = 20
plt.figure(figsize=(10,10))

for i in range(range_start,range_start + 10):
    plt.subplot(5,5,i-range_start+1)
    plt.xticks([])
    plt.xlabel(str(train_labels[i]))
    plt.imshow(train_images[i])
plt.show()

In [ ]:
train_images = train_images / 255.0
test_images = test_images / 255.0

In [ ]:
train_images[0].size

In [ ]:
model=None
model = tf.keras.Sequential()

model.add(tf.keras.layers.Flatten(input_shape=(28,28)))
model.add(tf.keras.layers.Dense(100,activation='relu'))
model.add(tf.keras.layers.Dense(10,activation='relu'))
model.add(tf.keras.layers.Dense(10,activation='softmax'))
model.summary()

In [ ]:
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.03),loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(train_images, train_labels, epochs=60,verbose=1,batch_size=1000,validation_split=0.3)


In [ ]:
loss, acc = model.evaluate(train_images,  train_labels, verbose=2)
print("\nTraining accuracy: ", acc)
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("\nTesting accuracy: ", acc)

In [ ]:
def plot_loss(history,data_names):
    n = 1
    plt.figure(n,figsize=(20,5))

    for i in data_names:
        plt.plot(history.history[i],label=i)
        plt.xlim([0, len(history.history[i])])
        plt.ylim([np.min(history.history[i]), np.max(history.history[i])])
        plt.xlabel('Epoch')
        #plt.ylabel(i)
        n += 1

    plt.legend()
    plt.grid(True)
    plt.show()

plot_loss(history,['loss','val_loss'])
plot_loss(history,['accuracy','val_accuracy'])



In [ ]:
#pred_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])
predictions = model.predict(test_images)

In [ ]:
def plot_image(i, predictions_array, true_label, img):
    true_label, img = true_label[i], img[i]
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])

    plt.imshow(img)

    predicted_label = np.argmax(predictions_array)
    if predicted_label == true_label:
        color = 'blue'
    else:
        color = 'red'

    plt.xlabel("{} {:2.0f}% ({})".format(str(predicted_label),
                                         100*np.max(predictions_array),
                                         str(true_label)),
               color=color)


def plot_value_array(i, predictions_array, true_label):
    true_label = true_label[i]
    plt.grid(False)
    plt.xticks(range(10))
    plt.yticks([])
    thisplot = plt.bar(range(10), predictions_array, color="#777777")
    plt.ylim([0, 1])
    predicted_label = np.argmax(predictions_array)

    thisplot[predicted_label].set_color('red')
    thisplot[true_label].set_color('blue')

In [ ]:
range_start = 100
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(range_start,range_start+num_images):
    j = i - range_start
    plt.subplot(num_rows, 2*num_cols, 2*j+1)
    plot_image(i, predictions[i], test_labels, test_images)
    plt.subplot(num_rows, 2*num_cols, 2*j+2)
    plot_value_array(i, predictions[i], test_labels)
plt.tight_layout()
plt.show()

In [ ]:
import tf2onnx
import onnx

input_signature = [tf.TensorSpec(model.inputs[0].shape, tf.float64, name='bitmap')]

onnx_model, _ = tf2onnx.convert.from_keras(model, input_signature, opset=13)
onnx.save(onnx_model, "model.onnx")

In [ ]:
%pip install onnxruntime
import onnxruntime
session = onnxruntime.InferenceSession("model.onnx")


In [ ]:
train_images[4:5].dtype

In [ ]:
inputs = {"bitmap":train_images[4:5]}

In [ ]:
inputs['bitmap'].dtype

In [ ]:
results = session.run([],inputs)